## 6.　モデルチューニング

- LightGBMのハイパーパラメータのチューニング
- scikit-learnのモデル利用
- ニューラルネットワークの利用
- アンサンブル

In [22]:
import numpy as np
import pandas as pd
import os
import pickle
import gc
# 分布の確認
import pandas_profiling as pdp
# 可視化
import matplotlib.pyplot as plt
# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
# モデリング
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

# matplotlibで日本語表示したい場合はこれをinstallしてインポートする
# !pip install japanize-matplotlib
# import japanize_matplotlib
# %matplotlib inline

df_train = pd.read_csv("../data/train.csv")
x_train, y_train, id_train = df_train[['Pclass', 'Fare']], df_train[[
    'Survived']], df_train[['PassengerId']]

### 6.1　LightGBMのハイパーパラメータのチューニング

##### 6.1.1　手動チューニング
1. 初期値の設定
2. 学習結果に応じた個別チューニング

##### 6.1.2　自動チューニング

optunaを用いた自動チューニングの例

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [12]:
!pip install optuna

You should consider upgrading via the 'c:\users\statistics\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [13]:
import optuna

In [23]:
# 詮索しないハイパーパラメータ
params_base = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric":"auc",
    "learning_rate": "0.02",
    "n_estimators": 100000,
    "bagging_fleq": 1,
    "seed": 123,
}

def objective(trial):
    # 詮索するハイパーパラメータ
    params_tuning = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 256),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 200),
        "min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e2, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e2, log=True),
    }
    params_tuning.update(params_base)

    # モデル学習・評価
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    for nfold in np.arange(5):
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
        x_va, y_va = x_train.loc[idx_va, :], y_train.loc[idx_va, :]
        model = lgb.LGBMClassifier(**params_tuning)
        model.fit(x_tr,
                  y_tr, 
                  eval_set=[(x_tr, y_tr), (x_va, y_va)],
                  early_stopping_rounds=100,
                  verbose=0,)
        y_va_pred = model.predict_proba(x_va)[:, 1]
        metric_va = accuracy_score(y_va, np.where(y_va_pred>=0.5, 1, 0))
        list_metrics.append(metric_va)

    # 評価値の計算
    metrics = np.mean(list_metrics)

    return metrics

In [ ]:
# 最適化処理（詮索の実行）
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction="maximize")
study.optimize(objective, n_trials=30)

In [36]:
# 詮索結果の確認
trial = study.best_trial
print("acc(best)={:,.4f}".format(trial.value))
print(f"acc(best)={trial.value}")
display(trial.params)

acc(best)=0.6992
acc(best)=0.6992404745464817


{'num_leaves': 252,
 'min_data_in_leaf': 60,
 'min_sum_hessian_in_leaf': 0.009739830877756862,
 'feature_fraction': 0.8018999555097835,
 'bagging_fraction': 0.5949431124260618,
 'lambda_l1': 0.1812977929299853,
 'lambda_l2': 0.011197876549499167}

In [38]:
# ベストなハイパーパラメータの取得
params_best = trial.params
params_best.update(params_base)
display(params_best)

{'num_leaves': 252,
 'min_data_in_leaf': 60,
 'min_sum_hessian_in_leaf': 0.009739830877756862,
 'feature_fraction': 0.8018999555097835,
 'bagging_fraction': 0.5949431124260618,
 'lambda_l1': 0.1812977929299853,
 'lambda_l2': 0.011197876549499167,
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'learning_rate': '0.02',
 'n_estimators': 100000,
 'bagging_fleq': 1,
 'seed': 123}

### 6.2　LightGBM以外のモデル利用

- scikit-learnの各種モデル
- ニューラルネットワーク

##### 6.2.1　scikit-learnの各種モデル

基本的にはどのモデルでも同じ手順で処理できる.

1. モデル定義: importした関数を指定してモデルを定義
2. 学習: .fitで学習を実行する
3. .predictで推論処理を実行する

sklearnの各種モデルでの学習の際の注意点
- 欠損値を埋めないと学習できない
- すべて数値データにしないと学習できない
- 数値データを正規化あるいは標準化する

##### ロジスティック回帰

簡単のため説明変数は'Pclass', 'Age', 'Embarked'の３つとする

In [40]:
# ファイルの読み込み
df_train = pd.read_csv("../data/train.csv")
# データセットの作成
x_train = df_train[['Pclass', 'Age', 'Embarked']]
y_train = df_train[['Survived']]

'Age', 'Embarked'には欠損値があるので, 欠損値を保管する

In [42]:
# 欠損値補間: 数値データ
x_train['Age'] = x_train['Age'].fillna(x_train['Age'].mean())
# 欠損値補間: カテゴリ変数
x_train["Embarked"] = x_train['Embarked'].fillna(x_train['Embarked'].mode()[0])

カテゴリ変数である'Embarked'をohe-hot-encodingで数値データに変換

In [44]:
# カテゴリ変数の数値データへの変換（one-hot-encoding）
ohe = OneHotEncoder()
ohe.fit(x_train[['Embarked']])
df_embarked = pd.DataFrame(ohe.transform(x_train[['Embarked']]).toarray(), columns=[f"Embarked_{col}" for col in ohe.categories_[0]])
x_train = pd.concat([x_train, df_embarked], axis=1)
x_train = x_train.drop(columns=['Embarked'])

In [47]:
# 数値データの正規化
mms = MinMaxScaler()
mms.fit(df_train[['Pclass']])
df_train['Pclass'] = mms.transform(df_train[['Pclass']])

mms.fit(df_train[['Age']])
df_train['Age'] = mms.transform(df_train[['Age']])

In [49]:
# 学習データと検証データの分割（ホールドアウト検証）
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_tr.shape, x_va.shape, y_tr.shape, y_va.shape)

(712, 5) (179, 5) (712, 1) (179, 1)


In [51]:
# モデル定義
from sklearn.linear_model import LogisticRegression
model_logis = LogisticRegression()

# 学習
model_logis.fit(x_tr, y_tr)

# 予測
y_va_pred = model_logis.predict(x_va)
print(f"accuracy: {accuracy_score(y_va, y_va_pred)}")
print(y_va_pred[:5])

# 確率値の取得
y_va_pred_prob = model_logis.predict_proba(x_va)
print(y_va_pred_prob[:5, :])

accuracy: 0.7262569832402235
[0 1 0 1 0]
[[0.85951356 0.14048644]
 [0.20358813 0.79641187]
 [0.85501264 0.14498736]
 [0.28727379 0.71272621]
 [0.61610234 0.38389766]]


##### SVM（サポートベクターマシン）
今回はSVMの分類モデル

In [55]:
# モデル定義
from sklearn.svm import SVC
model_svm = SVC(C=1.0, random_state=123, probability=True)

# 学習
model_svm.fit(x_tr, y_tr)

# 予測
y_va_pred = model_svm.predict(x_va)
print(f"accuracy: {accuracy_score(y_va, y_va_pred)}")
print(y_va_pred[:5])

# 確率値の取得
y_va_pred_prob = model_svm.predict_proba(x_va)
print(y_va_pred_prob[:5, :])


accuracy: 0.6368715083798883
[0 0 0 0 0]
[[0.66192458 0.33807542]
 [0.5599569  0.4400431 ]
 [0.66089667 0.33910333]
 [0.57490543 0.42509457]
 [0.58408982 0.41591018]]


##### 6.2.2　ニューラルネットワーク

ニューラルネットワークを用いて学習する際の注意点
- 欠損値を埋めないと学習できない
- すべて数値データにしないと学習できない
- 数値データを正規化あるいは標準化する